In [1]:
from tqdm.notebook import tqdm
import pandas as pd

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
DEV = False
# model_name = 'dunzhang/stella_en_400M_v5'
model_name = "avsolatorio/GIST-Embedding-v0" # train when I've got a spare two hours

: 

In [ ]:
def import_labelled_data(path="data/labelled/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data

data = import_labelled_data(path='../../data/labelled/data.json',)

if DEV:
    data = data.sample(5000)

# train test split
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

data.head()

: 

In [ ]:
from chunking import chunk_dataset_and_explode


# roughly 4 characters per token
max_len = 2048

train_data = chunk_dataset_and_explode(train_data, max_len=max_len, overlap=int(max_len * 0.2))
test_data = chunk_dataset_and_explode(test_data, max_len=max_len, overlap=int(max_len * 0.2))

: 

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data, split="train")
test_dataset = Dataset.from_pandas(test_data, split="test")

train_dataset

: 

In [ ]:
# embeddings
from sentence_transformers import SentenceTransformer


model = SentenceTransformer(model_name,trust_remote_code=True,similarity_fn_name='dot')

model.cuda()
# model.cpu()

: 

In [ ]:
model.device

: 

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
# get version of cuda
print(torch.version.cuda)
# clear cache
torch.cuda.empty_cache()

: 

In [ ]:
train_embeddings = model.encode(
    train_dataset["text"],
    show_progress_bar=True,
    normalize_embeddings=True,
)

: 

In [ ]:
test_embeddings = model.encode(
    test_dataset["text"],
    show_progress_bar=True,
    normalize_embeddings=True,
)

: 

In [ ]:
train_embeddings.shape

: 

In [ ]:
train_embeddings[0]

: 

In [ ]:
# add to pandas dataframe
train_data["embeddings"] = train_embeddings.tolist()
test_data["embeddings"] = test_embeddings.tolist()

: 

In [ ]:
train_data.head()

: 

In [ ]:
import os

# Create the folder if it doesn't exist
if not os.path.exists(f"embeddings/{model_name}/dev"):
    os.makedirs(f"embeddings/{model_name}/dev")

if DEV:
    train_data.to_json(f"embeddings/{model_name}/dev/train_embeddings.json", orient='records', indent=4)
    test_data.to_json(f"embeddings/{model_name}/dev/test_embeddings.json", orient='records', indent=4)
else:
    train_data.to_json(f"embeddings/{model_name}/train_embeddings.json", orient='records')
    test_data.to_json(f"embeddings/{model_name}/test_embeddings.json", orient='records')

    # also save dev data
    devTrain = train_data.sample(10000)
    devTest = test_data.sample(int(10000*0.2))
    devTrain.to_json(f"embeddings/{model_name}/dev/train_embeddings.json", orient='records')
    devTest.to_json(f"embeddings/{model_name}/dev/test_embeddings.json", orient='records')



: 